COCO Conversion

In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import cv2
import base64
import time
import math
import datetime
import os
import random
from PIL import Image
from io import BytesIO
from tqdm.notebook import tqdm
from glob import glob

from collections import defaultdict

base_dir = "/Users/ljj97/Downloads/open"

In [1]:
with open('/Users/ljj97/Downloads/open/classes.txt', 'r') as f:
        lines=f.readlines()

class_list=[]
for i in lines:
        class_list.append(i[:-2])

dictionary = {string : i+1 for i,string in enumerate(class_list)}
print(dictionary)

print(class_list)

{'000,chevrolet_malibu_sedan_2012_201': 1, '001,chevrolet_malibu_sedan_2017_201': 2, '002,chevrolet_spark_hatchback_2016_202': 3, '003,chevrolet_trailblazer_suv_2021': 4, '004,chevrolet_trax_suv_2017_201': 5, '005,genesis_g80_sedan_2016_202': 6, '006,genesis_g80_sedan_2021': 7, '007,genesis_gv80_suv_2020': 8, '008,hyundai_avante_sedan_2011_201': 9, '009,hyundai_avante_sedan_2020': 10, '010,hyundai_grandeur_sedan_2011_201': 11, '011,hyundai_grandstarex_van_2018_202': 12, '012,hyundai_ioniq_hatchback_2016_201': 13, '013,hyundai_sonata_sedan_2004_200': 14, '014,hyundai_sonata_sedan_2010_201': 15, '015,hyundai_sonata_sedan_2019_202': 16, '016,kia_carnival_van_2015_202': 17, '017,kia_carnival_van_2021': 18, '018,kia_k5_sedan_2010_201': 19, '019,kia_k5_sedan_2020': 20, '020,kia_k7_sedan_2016_202': 21, '021,kia_mohave_suv_2020': 22, '022,kia_morning_hatchback_2004_201': 23, '023,kia_morning_hatchback_2011_201': 24, '024,kia_ray_hatchback_2012_201': 25, '025,kia_sorrento_suv_2015_201': 26, '02

In [9]:
def convert_to_coco(
    label_paths, # 변경
    save_path,
):
    """
        only for train dataset
    """
    res = defaultdict(list)
    # 제거
    # json_paths = glob(os.path.join(root_path, 'train', '*.json'))
    categories = dictionary
    
    n_id = 0
    i=0
    for label_path in tqdm(label_paths):
        with open(label_path, 'r') as f:
            lines = f.readlines()
            
        
        image_id = ('{0:05d}'.format(i))
        res['images'].append({
            'id': int(image_id),
            'width': 1920,
            'height': 1040,
            'file_name': 'syn_'+str(image_id)+".png", # 변경
        })
        i+=1
        
        for line in lines:
            print(line)
            values = list(map(float, line.strip().split(' ')))
            class_id = int(values[0])
            x1, y1 = int(round(values[1])), int(round(values[2]))
            x2, y2 = int(round(max(values[3], values[5], values[7]))), int(round(max(values[4], values[6], values[8])))
            
            w, h = x2 - x1, y2 - y1
            
            res['annotations'].append({
                'id': n_id,
                'image_id': image_id,
                'category_id': class_id,
                'area': w * h,
                'bbox': [x1, y1, w, h],
                'iscrowd': 0,
            })
            n_id += 1
    
    for name, id in categories.items():
        res['categories'].append({
            'id': id,
            'name': name,
        })
        
    with open(save_path, 'w') as f:
        json.dump(res, f)

In [2]:
from sklearn.model_selection import train_test_split
from glob import glob
from pathlib import Path

image_path=Path('/Users/ljj97/Downloads/open/train')

train_files = list(image_path.glob('*.txt'))

train_path_list,valid_path_list = train_test_split(train_files,test_size=0.2,random_state=42)

In [21]:
import cv2
from tqdm import tqdm

#for address in tqdm(train_path_list):
 #   a=i[:-4]+'.png'
  #  image=cv2.imread(a)
   # cv2.imwrite('/Users/ljj97/Desktop/coco/train2017/'+a[-13:], image)

for address in tqdm(valid_path_list):
    i=str(address)
    a=i[:-4]+'.png'
    image=cv2.imread(i[:-4]+'.png')
    cv2.imwrite('/Users/ljj97/Desktop/coco/val2017/'+a[-13:], image)

100%|██████████| 1297/1297 [02:15<00:00,  9.56it/s]


In [3]:
import cv2
from tqdm import tqdm
from glob import glob
from pathlib import Path

test_path=Path('/Users/ljj97/Downloads/open/test')

test_files = list(test_path.glob('*.png'))

for address in tqdm(test_files):
    i=str(address)
    a=i[:-4]+'.png'
    image=cv2.imread(i[:-4]+'.png')
    cv2.imwrite('/Users/ljj97/Desktop/coco/test2017/'+a[-13:-4]+'.jpg', image)

100%|██████████| 3400/3400 [03:17<00:00, 17.25it/s]


In [12]:
convert_to_coco(train_path_list, os.path.join(base_dir, 'train_annotations.json'))

  0%|          | 0/5184 [00:00<?, ?it/s]

28.0 1088 263 1380 263 1380 555 1088 555

16.0 671 154 957 154 957 473 671 473

26.0 328 241 604 241 604 569 328 569

7.0 1007 207 1312 207 1312 517 1007 517

2.0 638 175 859 175 859 421 638 421

16.0 327 414 632 414 632 794 327 794

19.0 1109 427 1420 427 1420 716 1109 716

8.0 767 515 1055 515 1055 811 767 811

1.0 313 387 588 387 588 672 313 672

9.0 1096 407 1395 407 1395 680 1096 680

29.0 627 176 865 176 865 426 627 426

19.0 340 500 623 500 623 807 340 807

27.0 1061 315 1345 315 1345 610 1061 610

12.0 738 467 1035 467 1035 767 738 767

29.0 309 333 551 333 551 608 309 608

7.0 1117 350 1455 350 1455 693 1117 693

1.0 653 228 932 228 932 488 653 488

29.0 360 448 623 448 623 751 360 751

17.0 705 315 1033 315 1033 676 705 676

4.0 1104 318 1384 318 1384 619 1104 619

18.0 651 205 911 205 911 456 651 456

3.0 356 428 634 428 634 755 356 755

9.0 1056 277 1336 277 1336 525 1056 525

4.0 744 331 1007 331 1007 631 744 631

27.0 381 440 648 440 648 761 381 761

16.0 974 179 1275 179

In [13]:
convert_to_coco(valid_path_list, os.path.join(base_dir, 'val_annotations.json'))

  0%|          | 0/1297 [00:00<?, ?it/s]

11.0 1016 173 1325 173 1325 529 1016 529

1.0 684 314 970 314 970 588 684 588

24.0 355 392 597 392 597 712 355 712

32.0 1051 231 1337 231 1337 521 1051 521

33.0 785 441 1073 441 1073 751 785 751

3.0 369 426 648 426 648 752 369 752

7.0 1128 465 1474 465 1474 822 1128 822

16.0 698 378 1013 378 1013 738 698 738

2.0 334 449 568 449 568 733 334 733

29.0 1189 484 1490 484 1490 784 1189 784

31.0 780 476 1062 476 1062 757 780 757

29.0 408 560 677 560 677 873 408 873

5.0 1125 370 1436 370 1436 654 1125 654

26.0 807 452 1135 452 1135 820 807 820

27.0 417 510 692 510 692 841 417 841

17.0 1005 188 1321 188 1321 512 1005 512

18.0 739 410 1023 410 1023 695 739 695

28.0 334 322 587 322 587 619 334 619

25.0 1081 296 1391 296 1391 614 1081 614

23.0 677 214 900 214 900 453 677 453

5.0 353 401 622 401 622 690 353 690

1.0 647 254 923 254 923 514 647 514

12.0 271 211 515 211 515 460 271 460

1.0 638 156 905 156 905 403 638 403

27.0 317 230 561 230 561 517 317 517

26.0 990 152 1290 15

MMDetection

In [ ]:
!pip show torch

In [ ]:
# latest installation with pytorch Ref : https://mmcv.readthedocs.io/en/latest/get_started/installation.html
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.10.0/index.html


# Install mmdetection
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection

!pip install -e .

# install Pillow 7.0.0 back in order to avoid bug in colab
!pip install Pillow==7.0.0

In [ ]:
#Runtime Restart

In [ ]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

In [ ]:
%cd /Users/ljj97/mmdetection
!mkdir checkpoints
!wget -c https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth \
      -O checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth

In [6]:
%%writefile configs/cascade_rcnn/custom_cascade_rcnn_r101_fpn_1x_coco.py

_base_ = '../cascade_rcnn/cascade-rcnn_r101_fpn_1x_coco.py'

# We also need to change the num_classes in head to match the dataset's annotation

model = dict(
    roi_head=dict(
        bbox_head=[ dict(
                type='Shared2FCBBoxHead',
                in_channels=256,
                fc_out_channels=1024,
                roi_feat_size=7,
                num_classes=34,
                bbox_coder=dict(
                    type='DeltaXYWHBBoxCoder',
                    target_means=[0., 0., 0., 0.],
                    target_stds=[0.1, 0.1, 0.2, 0.2]),
                reg_class_agnostic=True,
                loss_cls=dict(
                    type='CrossEntropyLoss',
                    use_sigmoid=False,
                    loss_weight=1.0),
                loss_bbox=dict(type='SmoothL1Loss', beta=1.0,
                               loss_weight=1.0)),
            dict(
                type='Shared2FCBBoxHead',
                in_channels=256,
                fc_out_channels=1024,
                roi_feat_size=7,
                num_classes=34,
                bbox_coder=dict(
                    type='DeltaXYWHBBoxCoder',
                    target_means=[0., 0., 0., 0.],
                    target_stds=[0.05, 0.05, 0.1, 0.1]),
                reg_class_agnostic=True,
                loss_cls=dict(
                    type='CrossEntropyLoss',
                    use_sigmoid=False,
                    loss_weight=1.0),
                loss_bbox=dict(type='SmoothL1Loss', beta=1.0,
                               loss_weight=1.0)),
            dict(
                type='Shared2FCBBoxHead',
                in_channels=256,
                fc_out_channels=1024,
                roi_feat_size=7,
                num_classes=34,
                bbox_coder=dict(
                    type='DeltaXYWHBBoxCoder',
                    target_means=[0., 0., 0., 0.],
                    target_stds=[0.033, 0.033, 0.067, 0.067]),
                reg_class_agnostic=True,
                loss_cls=dict(
                    type='CrossEntropyLoss',
                    use_sigmoid=False,
                    loss_weight=1.0),
                loss_bbox=dict(type='SmoothL1Loss', beta=1.0, loss_weight=1.0))],
        mask_head=dict(
            type='FCNMaskHead',
            num_convs=4,
            in_channels=256,
            conv_out_channels=256,
            num_classes=34,
            loss_mask=dict(
                type='CrossEntropyLoss', use_mask=True, loss_weight=1.0))))


# Modify dataset related settings
data_root = '/Users/ljj97/Desktop/coco'
metainfo = {
    'classes': ('000,chevrolet_malibu_sedan_2012_201', '001,chevrolet_malibu_sedan_2017_201', '002,chevrolet_spark_hatchback_2016_202', '003,chevrolet_trailblazer_suv_2021', '004,chevrolet_trax_suv_2017_201', '005,genesis_g80_sedan_2016_202', '006,genesis_g80_sedan_2021', '007,genesis_gv80_suv_2020', '008,hyundai_avante_sedan_2011_201', '009,hyundai_avante_sedan_2020', '010,hyundai_grandeur_sedan_2011_201', '011,hyundai_grandstarex_van_2018_202', '012,hyundai_ioniq_hatchback_2016_201', '013,hyundai_sonata_sedan_2004_200', '014,hyundai_sonata_sedan_2010_201', '015,hyundai_sonata_sedan_2019_202', '016,kia_carnival_van_2015_202', '017,kia_carnival_van_2021', '018,kia_k5_sedan_2010_201', '019,kia_k5_sedan_2020', '020,kia_k7_sedan_2016_202', '021,kia_mohave_suv_2020', '022,kia_morning_hatchback_2004_201', '023,kia_morning_hatchback_2011_201', '024,kia_ray_hatchback_2012_201', '025,kia_sorrento_suv_2015_201', '026,kia_sorrento_suv_2020', '027,kia_soul_suv_2014_201', '028,kia_sportage_suv_2016_202', '029,kia_stonic_suv_2017_201', '030,renault_sm3_sedan_2015_201', '031,renault_xm3_suv_2020', '032,ssangyong_korando_suv_2019_202', '033,ssangyong_tivoli_suv_2016_202', ),
    'palette': [
        (220, 20, 60),(220, 20, 60),(220, 20, 60),(220, 20, 60),(220, 20, 60),
        (220, 20, 60),(220, 20, 60),(220, 20, 60),(220, 20, 60),(220, 20, 60),
        (220, 20, 60),(220, 20, 60),(220, 20, 60),(220, 20, 60),(220, 20, 60),
        (220, 20, 60),(220, 20, 60),(220, 20, 60),(220, 20, 60),(220, 20, 60),
        (220, 20, 60),(220, 20, 60),(220, 20, 60),(220, 20, 60),(220, 20, 60),
        (220, 20, 60),(220, 20, 60),(220, 20, 60),(220, 20, 60),(220, 20, 60),
        (220, 20, 60),(220, 20, 60),(220, 20, 60),(220, 20, 60)
    ]
}
train_dataloader = dict(
    batch_size=16,
    dataset=dict(
        data_root=data_root,
        metainfo=metainfo,
        ann_file='annotations/annotations_train2017.json',
        data_prefix=dict(img='train2017/')))
val_dataloader = dict(
    dataset=dict(
        data_root=data_root,
        metainfo=metainfo,
        ann_file='annotations/annotations_val2017.json',
        data_prefix=dict(img='val2017/')))
test_dataloader = val_dataloader

# Modify metric related settings
val_evaluator = dict(ann_file=data_root + 'val/annotations_val2017.json')
test_evaluator = val_evaluator

# We can use the pre-trained Mask RCNN model to obtain higher performance
load_from = 'https://download.openmmlab.com/mmdetection/v2.0/cascade_rcnn/cascade_rcnn_x101_32x4d_fpn_1x_coco/cascade_rcnn_x101_32x4d_fpn_1x_coco_20200316-95c2deb6.pth'

Overwriting configs/cascade_rcnn/custom_cascade_rcnn_r101_fpn_1x_coco.py


In [ ]:
!python tools/train.py configs/cascade_rcnn/custom_cascade_rcnn_r101_fpn_1x_coco.py

In [ ]:
# Choose to use a config and initialize the detector
config = "configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py"
# Setup a checkpoint file to load
checkpoint = '/checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'
# initialize the detector
#model = init_detector(config, checkpoint, device='cuda:0')

Training

In [ ]:
import mmcv
import matplotlib.pyplot as plt

import copy
import os.path as osp

import mmcv
import numpy as np

In [ ]:
from mmcv import Config
cfg = Config.fromfile(config)

In [ ]:
from mmdet.apis import set_random_seed

base_path = "/Users/ljj97/Downloads/open/" # base_dir과 같습니다.

test_anno = "valid_annotations.json" # 출력은 "valid_partial_annotations.json"
train_anno = "train_annotations.json" # 출력은 "train_partial_annotations.json"

test_img =  "train_img" # 출력은 "train_100000"
train_img = "train_img" # 출력은 "train_100000"


save_dir = os.path.join(base_path, "work_dir")
if not os.path.exists(save_path_dir):
    os.makedirs(save_path_dir)

# Modify dataset type and path
cfg.dataset_type = 'CocoDataset'
cfg.data_root = base_path

cfg.data.test.type = 'CocoDataset'
cfg.data.test.data_root = base_path
cfg.data.test.ann_file = test_anno
cfg.data.test.img_prefix = test_img

cfg.data.train.type = 'CocoDataset'
cfg.data.train.data_root = base_path
cfg.data.train.ann_file = train_anno
cfg.data.train.img_prefix = train_img

cfg.data.val.type = 'CocoDataset'
cfg.data.val.data_root = base_path
cfg.data.val.ann_file = test_anno
cfg.data.val.img_prefix = test_img

cfg.data.samples_per_gpu = 8
cfg.data.workers_per_gpu = 2

classes = ('01_ulcer', '02_mass', '04_lymph', '05_bleeding')

cfg.data.train.classes = classes
cfg.data.val.classes = classes
cfg.data.test.classes = classes


# modify num classes of the model in box head
cfg.model.roi_head.bbox_head.num_classes = 4
# We can still use the pre-trained Mask RCNN model though we do not need to
# use the mask branch
cfg.load_from = "./checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth" # Error가 날 경우, "/content/mmdetection/checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth"

# Set up working dir to save files and logs.
cfg.work_dir = save_dir

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.optimizer.lr = 0.02 / 8
cfg.lr_config.warmup = None

# 에폭 수 조절
cfg.runner = dict(type='EpochBasedRunner', max_epochs=2)

cfg.log_config.interval = 2

# Change the evaluation metric since we use customized dataset.
cfg.evaluation.metric = 'mAP'
# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 4
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 4

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)


# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

In [ ]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector


# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

Submission

In [ ]:
results = {
    'file_name':[], 'class_id':[], 'confidence':[], 'point1_x':[], 'point1_y':[],
    'point2_x':[], 'point2_y':[], 'point3_x':[], 'point3_y':[], 'point4_x':[], 'point4_y':[]
}

score_threshold = 0.4 # 0.8, 0.3, cfg.model.test_cfg.rcnn.score_thr
   
for index, img_path in tqdm(enumerate(test_file), total = len(test_file)):
    
    file_name = img_path.split("/")[-1].split(".")[0]+".json"

    img = mmcv.imread(img_path)
    predictions = inference_detector(model, img)
    boxes, scores, labels = (list(), list(), list())

    for k, cls_result in enumerate(predictions):
        # print("cls_result", cls_result)
        if cls_result.size != 0:
            if len(labels)==0:
                boxes = np.array(cls_result[:, :4])
                scores = np.array(cls_result[:, 4])
                labels = np.array([k+1]*len(cls_result[:, 4]))
            else:    
                boxes = np.concatenate((boxes, np.array(cls_result[:, :4])))
                scores = np.concatenate((scores, np.array(cls_result[:, 4])))
                labels = np.concatenate((labels, [k+1]*len(cls_result[:, 4])))

    if len(labels) != 0:
        indexes = np.where(scores > score_threshold)
        # print(indexes)
        boxes = boxes[indexes]
        scores = scores[indexes]
        labels = labels[indexes]

        for label, score, bbox in zip(labels, scores, boxes):
            x_min, y_min, x_max, y_max = bbox.astype(np.int64)

            results['file_name'].append(file_name)
            results['class_id'].append(label)
            results['confidence'].append(score)
            results['point1_x'].append(x_min)
            results['point1_y'].append(y_min)
            results['point2_x'].append(x_max)
            results['point2_y'].append(y_min)
            results['point3_x'].append(x_max)
            results['point3_y'].append(y_max)
            results['point4_x'].append(x_min)
            results['point4_y'].append(y_max)

In [ ]:
import pandas as pd
submission = pd.DataFrame(results)
submission.shape